## Dimer scoring
This notebook from [github.com/matteoferla/pyrosetta_help](https://github.com/matteoferla/pyrosetta_help).

It can be opened in Colabs via [https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta-dimer.ipynb](https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta-dimer.ipynb)

This notebook is intened for use with a [ColabFold output](https://github.com/sokrypton/ColabFold) from a complex of two protein.
It loads, relaxes (=energy minises) and scores the interface of these.
Some tricks discussed in https://blog.matteoferla.com/2021/08/tweaking-alphafold2-models-with.html

#### Resource location
This notebook is from the repository [pyrosetta help](https://github.com/matteoferla/pyrosetta_help)
<!--- silly badges: --->
[![https img shields io github forks matteoferla pyrosetta_help label Fork style social logo github](https://img.shields.io/github/forks/matteoferla/pyrosetta_help?label=Fork&style=social&logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github stars matteoferla pyrosetta_help style social logo github](https://img.shields.io/github/stars/matteoferla/pyrosetta_help?style=social&logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github watchers matteoferla pyrosetta_help label Watch style social logo github](https://img.shields.io/github/watchers/matteoferla/pyrosetta_help?label=Watch&style=social&logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github last commit matteoferla pyrosetta_help logo github](https://img.shields.io/github/last-commit/matteoferla/pyrosetta_help?logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github license matteoferla pyrosetta_help logo github](https://img.shields.io/github/license/matteoferla/pyrosetta_help?logo=github)](https://github.com/matteoferla/pyrosetta_help/raw/master/LICENCE)
[![https img shields io github release date matteoferla pyrosetta_help logo github](https://img.shields.io/github/release-date/matteoferla/pyrosetta_help?logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github commit activity m matteoferla pyrosetta_help logo github](https://img.shields.io/github/commit-activity/m/matteoferla/pyrosetta_help?logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github issues matteoferla pyrosetta_help logo github](https://img.shields.io/github/issues/matteoferla/pyrosetta_help?logo=github)](https://github.com/matteoferla/pyrosetta_help)
[![https img shields io github issues closed matteoferla pyrosetta_help logo github](https://img.shields.io/github/issues-closed/matteoferla/pyrosetta_help?logo=github)](https://github.com/matteoferla/pyrosetta_help)

#### Authors, funding and affiliations
_Matteo Ferla_: Taylor group / Oxford Genomic Medicine theme, Wellcome Centre for Human Genetics, University of Oxford
[WCHG](https://www.well.ox.ac.uk/people/matteo-ferla)
[![https img shields io badge orcid 0000 0002 5508 4673 a6ce39 logo orcid](https://img.shields.io/badge/orcid-0000--0002--5508--4673-a6ce39?logo=orcid)](https://orcid.org/0000--0002--5508--4673) [![https img shields io badge google scholar gF bp_cAAAAJ success logo googlescholar](https://img.shields.io/badge/google--scholar-gF--bp_cAAAAJ-success?logo=googlescholar)](https://scholar.google.com/citations?user=gF--bp_cAAAAJ&hl=en) [![https img shields io twitter follow matteoferla label Follow logo twitter](https://img.shields.io/twitter/follow/matteoferla?label=Follow&logo=twitter)](https://twitter.com/matteoferla) [![https img shields io stackexchange stackoverflow r 4625475 logo stackoverflow](https://img.shields.io/stackexchange/stackoverflow/r/4625475?logo=stackoverflow)](https://stackoverflow.com/users/4625475) [![https img shields io stackexchange bioinformatics r 6322 logo stackexchange](https://img.shields.io/stackexchange/bioinformatics/r/6322?logo=stackexchange)](https://bioinformatics.stackexchange.com/users/6322) [![https img shields io badge email gmail informational logo googlemail](https://img.shields.io/badge/email-gmail-informational&logo=googlemail)](https://mailhide.io/e/Ey3RNO2G) [![https img shields io badge email Oxford informational logo googlemail](https://img.shields.io/badge/email-Oxford-informational&logo=googlemail)](https://mailhide.io/e/Y1dbgyyE)
![Ox](https://upload.wikimedia.org/wikipedia/en/thumb/2/2f/University_of_Oxford.svg/132px-University_of_Oxford.svg.png)

In [ ]:
#@title Installation
#@markdown Press the play button on the top right hand side of this cell
#@markdown once you have checked the settings.
#@markdown You will be notified that this notebook is not from Google, that is normal.
!pip install rdkit-pypi rdkit-to-params biopython pyrosetta-help
!pip install --upgrade plotly

import os
from importlib import reload
import pyrosetta_help as ph
from typing import *

# Muppet-proofing: are we in colab?
assert ph.get_shell_mode() == 'colab', 'This is a colab notebook, if running in Jupyter notebook, the installation is different'

# ============================================================================
#@markdown ### Use Google Drive
#@markdown Optionally store your results in your google drive.
#@markdown If `use_drive` is True, you will be prompted to give permission to use Google Drive
#@markdown (you may be prompted to follow a link and possibly authenticate and then copy a code into a box)
#@markdown —**_always_ remember to check strangers' code against data theft**:
#@markdown e.g. search and look for all instances of `http`, `requests` and `post` in the code, and
#@markdown make sure the creator is not typosquatting as someone else (e.g. username `Coogle`).
use_drive = True  #@param {type:"boolean"}
ph.mount_google_drive(use_drive)

# ============================================================================
#@markdown ### PyRosetta installation
#@markdown This will install PyRosetta in this Colab notebook (~2–15 minutes depending on time of day),
#@markdown but you will require a [PyRosetta licence](https://www.pyrosetta.org/home/licensing-pyrosetta)
#@markdown (free for academics).
#@markdown to speed things up _next_ time you can download a release into your Google Drive.
#@markdown Use Google Drive for PyRosetta:

#@markdown &#x1F44D; maybe faster next time

#@markdown &#x1F44E; occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

download_to_drive = False #@param {type:"boolean"}
download_path = '.' #@param {type:"string"}
#@markdown If this is the next-time, `download_to_drive` and the credentials below will be ignored if
#@markdown there's a release in `download_path`.

#@markdown The following is not the real username and password. However, the format is similar.
username = 'boltzmann'  #@param {type:"string"}
username.strip().lower()
password = 'constant'  #@param {type:"string"}
#@markdown If yours are not the the academic credentials
#@markdown disable this:
hash_comparison_required = True #@param {type:"boolean"}
#@markdown &#128544; THIS FLAG IS NOT PREVENTING YOU FROM USING PLAIN ROSETTA CREDENTIALS
#@markdown AS THE CUSTOM ERROR SAYS! **REGULAR ROSETTA CREDENTIALS DO NOT WORK FOR PYROSETTA.**

if download_to_drive and not use_drive:
    raise ValueError('You said False to `use_drive` and True to `download_to_drive`? Very funny.')
elif download_to_drive:
    ph.download_pyrosetta(username=username,
                          password=password,
                          path=download_path,
                          hash_comparison_required=hash_comparison_required)
else:
    pass

ph.install_pyrosetta(username=username,
                     password=password,
                     path=download_path,
                     hash_comparison_required=hash_comparison_required)
reload(ph)

# ??????? NOTE ??????????????????????????????????????????????????????????????????????
# ? Note to code spies
# ? this is a convoluted way to install pyrosetta via pyrosetta_help, due to options.
# ? the quicker way is:
# ?
# ? >>> pip install pyrosetta-help
# ? >>> install_pyrosetta -u xxx -p xxx
# ??????????????????????????????????????????????????????????????????????????????????

# disable as appropriate:
!pip install py3Dmol
# or:
!pip install nglview
from google.colab import output  # noqa (It's a colaboratory specific repo)
output.enable_custom_widget_manager()



# refresh imports
import site
site.main()

In [ ]:
#@title Start PyRosetta
import pyrosetta
import pyrosetta_help as ph

#@markdown Do not optimise hydrogen on loading:
no_optH = False #@param {type:"boolean"}
#@markdown Ignore (True) or raise error (False) if novel residue (e.g. ligand)
ignore_unrecognized_res=False  #@param {type:"boolean"}
#@markdown Use autogenerated PDB residues are often weird (bad geometry, wrong match, protonated etc.): —best do it properly and parameterise it
load_PDB_components=False  #@param {type:"boolean"}
#@markdown Ignore all waters:
ignore_waters=False  #@param {type:"boolean"}

extra_options= ph.make_option_string(no_optH=no_optH,
                                  ex1=None,
                                  ex2=None,
                                  mute='all',
                                  ignore_unrecognized_res=ignore_unrecognized_res,
                                  load_PDB_components=load_PDB_components,
                                  ignore_waters=ignore_waters)


# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)


In [ ]:
#@title Load pose
folder_name =  'prediction_38cac' #@param {type:"string"}
#@markdown This will create an object called `analyser` 
#@markdown which has the attributes
#@markdown * `scores` (pandas DataFrame),
#@markdown * three dictionaries (`original_poses`, `relaxed_poses`, `phospho_poses`)
#@markdown where the key is the rank number of the pose (value)
#@markdown * `errors` a dictionary with key = rank number and values are the matrices that 

 to be a Rosetta pose vector (without returning a clone)
analyser = ph.AF2NotebookAnalyser(folder=folder_name, load_poses=True)

In [ ]:
#@title Energy minimise

#@markdown Check to constraints the C&alpha; atoms of residues with a harmonic function with AF2 the pairwise error as sd.
#@markdown This is applied to two case:
#@markdown * all non–primary-sequence–adjecent residues with an error less than 12 &Aring; and 
#@markdown * residues in different chains that are less than 15 &Aring; apart ([Rosetta-style neighbouring distance](https://blog.matteoferla.com/2020/06/love-thy-neighbours-but-select-them.html), ~C&beta; distance)

constrained_errors=True  #@param {type:"boolean"}

#@markdown Number of FastRelax cycles

cycles=3   #@param {type:"integer"}
analyser.sidechain_relax(cycles) # prevent blowing up...
if constrained_errors:
 analyser.constrain(tolerance=2)
analyser.relax(cycles)
analyser.calculate_interface()

for i in analyser.errors:
    analyser.relaxed_poses[i].pdb_info( analyser.original_poses[i].pdb_info() )
    if i != 1:
        ph.superimpose_by_pLDDT(analyser.relaxed_poses[i], analyser.relaxed_poses[1])

In [ ]:
# show Pandas table:
analyser.scores

In [ ]:
#@title Add PTMs (optional)
#@markdown This is rather weird because:
#@markdown * PhosphoSitePlus does not have an API.
#@markdown * Colabs does not have a multiline input
#@markdown * this [way to copypaste data into a remove Jupyter Notebook](bhttps://gist.github.com/matteoferla/6de39f0057d283d270f834d487e358a9) does not work in Colabs
#@markdown So for now just copy paste the PTMs table in the next cell instead of `PASTE HERE`


In [ ]:
raw = '''PASTE HERE'''

In [ ]:
#@title Add PTMs
chain = 'A'  #@param {type:"string"}
pdb_ptms = analyser.parse_phosphosite(raw, maximum=analyser.original_poses[1].chain_end(1))
analyser.make_phosphorylated(pdb_ptms, chain, cycles)

In [ ]:
#@title Save poses and pandas dataframe

new_folder_name =  'output' #@param {type:"string"}
analyser.dump(new_folder_name)

In [ ]:
# Note that nglview does not work with Colabs but py3Dmol does.
# install py3Dmol
os.system(f'pip3 install py3Dmol')
import site
site.main()
# run
import py3Dmol
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(ph.get_pdbstr(analyser.relaxed_pose[1]),'pdb')
view.zoomTo()
view

In [ ]:
#@title Upload to Michelanglo (optional)
#@markdown [Michelanglo](https://michelanglo.sgc.ox.ac.uk/) is a website that
#@markdown allows the creation, annotation and sharing of a webpage with an interactive protein viewport.
#@markdown ([examples](https://michelanglo.sgc.ox.ac.uk/gallery)).
#@markdown The created pages are private —they have a 1 in a quintillion change to be guessed within 5 tries.

#@markdown Registered users (optional) can add interactive annotations to pages.
#@markdown Leave blank for guest:

username = ''  #@param {type:"string"}
password = ''  #@param {type:"string"}

#@markdown Choose initial model.

chosen_pose_series = 'relaxed' #@param ['original', 'relaxed', 'phospho'] {type:"string"}
chosen_rank = 1  #@param {type:"integer"}

os.system(f'pip3 install michelanglo-api')
import site
site.main()
from michelanglo_api import MikeAPI
if not username:
  mike = MikeAPI.guest_login()
else:
  mike = MikeAPI(username, password)

pose = {'original': analyser.original_poses, 
        'relaxed': analyser.relaxed_poses,
        'phospho': analyser.phospho_poses}[chosen_pose_series][chosen_rank]
page = mike.convert_pdb(pdbblock=ph.get_pdbstr(pose))
page.show_link()